In [3]:
#Load both the csv file from Part 1 pr.data.0.Current and the json file from Part 2 as dataframes (Spark, Pyspark, Pandas, Koalas, etc).

# Imports
import pandas as pd
import requests

# Data sources
currentData = "https://rearcquestutsav.s3.amazonaws.com/pr.data.0.Current"
populationData = "https://rearcquestutsav.s3.amazonaws.com/population.json"

# Load data
series = pd.read_csv(currentData, delimiter="\t")
print(series)
r = requests.get(populationData).json()
population = pd.json_normalize(r, record_path="data")
print(population)


       series_id          year period         value footnote_codes
0      PRS30006011        1995    Q01         2.600            NaN
1      PRS30006011        1995    Q02         2.100            NaN
2      PRS30006011        1995    Q03         0.900            NaN
3      PRS30006011        1995    Q04         0.100            NaN
4      PRS30006011        1995    Q05         1.400            NaN
...                  ...   ...    ...           ...            ...
34864  PRS88003203        2022    Q03       128.153            NaN
34865  PRS88003203        2022    Q04       129.072            NaN
34866  PRS88003203        2022    Q05       126.906            NaN
34867  PRS88003203        2023    Q01       131.196            NaN
34868  PRS88003203        2023    Q02       131.556            NaN

[34869 rows x 5 columns]
  ID Nation         Nation  ID Year  Year  Population    Slug Nation
0   01000US  United States     2020  2020   326569308  united-states
1   01000US  United States     2

In [4]:
#Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.
population1318 = population[(population["Year"].astype(int) >= 2013) &
                              (population["Year"].astype(int) <= 2018)]
population1318["Population"].mean(), population1318["Population"].std()

(317437383.0, 4257089.5415293295)

In [6]:
#Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year

max_value_series = series.groupby(["series_id        ", "year"], as_index=False)["       value"].agg("sum")
max_value_series = max_value_series.sort_values("       value", ascending=False).drop_duplicates("series_id        ", keep="first").sort_index().reset_index(drop=True)
max_value_series

,series_id,year,value
0,PRS30006011,2022,21.400
1,PRS30006012,2022,17.600
2,PRS30006013,1998,733.348
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,285.200
278,PRS88003193,2022,701.185
279,PRS88003201,2022,46.600
280,PRS88003202,2022,39.700


In [10]:
#Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset)
# Filter series data
specific_series = series.loc[series["series_id        "].str.contains("PRS30006032", case=False)]
specific_series = specific_series[specific_series["period"].str.contains("Q01", case=False)]
# Filter population data
population_extract = population["Population"].astype(int)
population["Year"] = population["Year"].astype(int)
# Merge and filter results
result = pd.merge(specific_series, population, left_on="year", right_on="Year", how="left")
result[["series_id        ", "year", "period", "       value", "Population"]]

,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.4,NaN
2,PRS30006032,1997,Q01,2.7,NaN
3,PRS30006032,1998,Q01,1.0,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.1,NaN
6,PRS30006032,2001,Q01,-6.0,NaN
7,PRS30006032,2002,Q01,-7.0,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.4,NaN
